In [1]:
#importing libraries
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

matches = pd.read_csv("C:\\Users\\moham\\Downloads\\data.csv")
deliveries = pd.read_csv("C:\\Users\\moham\\Downloads\\deliveries.csv")

In [3]:
matches.sample(10)

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
450,451,2014,Bangalore,2014-05-24,Royal Challengers Bangalore,Chennai Super Kings,Chennai Super Kings,field,normal,0,Chennai Super Kings,0,8,MS Dhoni,M Chinnaswamy Stadium,AK Chaudhary,NJ Llong,NaN
526,527,2016,Chandigarh,2016-04-17,Rising Pune Supergiants,Kings XI Punjab,Rising Pune Supergiants,bat,normal,0,Kings XI Punjab,0,6,M Vohra,"Punjab Cricket Association IS Bindra Stadium, Mohali",S Ravi,C Shamshuddin,NaN
148,149,2010,Kolkata,2010-04-04,Kolkata Knight Riders,Kings XI Punjab,Kolkata Knight Riders,bat,normal,0,Kings XI Punjab,0,8,DPMD Jayawardene,Eden Gardens,S Asnani,DJ Harper,NaN
224,225,2011,Chandigarh,2011-05-08,Kings XI Punjab,Pune Warriors,Kings XI Punjab,bat,normal,0,Pune Warriors,0,5,R Sharma,"Punjab Cricket Association Stadium, Mohali",SK Tarapore,RJ Tucker,NaN
236,237,2011,Dharamsala,2011-05-17,Kings XI Punjab,Royal Challengers Bangalore,Kings XI Punjab,bat,normal,0,Kings XI Punjab,111,0,AC Gilchrist,Himachal Pradesh Cricket Association Stadium,Asad Rauf,AM Saheba,NaN
209,210,2011,Kochi,2011-04-30,Delhi Daredevils,Kochi Tuskers Kerala,Delhi Daredevils,bat,normal,0,Delhi Daredevils,38,0,V Sehwag,Nehru Stadium,HDPK Dharmasena,AL Hill,NaN
562,563,2016,Chandigarh,2016-05-15,Kings XI Punjab,Sunrisers Hyderabad,Kings XI Punjab,bat,normal,0,Sunrisers Hyderabad,0,7,HM Amla,"Punjab Cricket Association IS Bindra Stadium, Mohali",KN Ananthapadmanabhan,M Erasmus,NaN
16,17,2008,Delhi,2008-04-30,Delhi Daredevils,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Delhi Daredevils,10,0,GD McGrath,Feroz Shah Kotla,Aleem Dar,I Shivram,NaN
278,279,2012,Jaipur,2012-04-23,Royal Challengers Bangalore,Rajasthan Royals,Rajasthan Royals,field,normal,0,Royal Challengers Bangalore,46,0,AB de Villiers,Sawai Mansingh Stadium,Asad Rauf,S Asnani,NaN
290,291,2012,Bangalore,2012-05-02,Royal Challengers Bangalore,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,4,Azhar Mahmood,M Chinnaswamy Stadium,BF Bowden,C Shamshuddin,NaN


In [4]:
deliveries.sample(10)

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
25447,109,1,Rajasthan Royals,Kolkata Knight Riders,9,1,NK Patel,YK Pathan,SS Sarkar,0,0,0,0,0,0,1,0,1,NaN,NaN,NaN
11239,49,2,Kings XI Punjab,Deccan Chargers,9,2,SE Marsh,Yuvraj Singh,Y Venugopal Rao,0,0,0,0,0,0,1,0,1,NaN,NaN,NaN
114691,483,1,Chennai Super Kings,Kings XI Punjab,3,5,DR Smith,BB McCullum,Karanveer Singh,0,0,0,0,0,0,2,0,2,NaN,NaN,NaN
107327,452,1,Sunrisers Hyderabad,Kolkata Knight Riders,15,5,DJG Sammy,Y Venugopal Rao,Shakib Al Hasan,0,0,0,0,0,0,1,0,1,NaN,NaN,NaN
56065,237,2,Royal Challengers Bangalore,Kings XI Punjab,4,1,V Kohli,SS Tiwary,RJ Harris,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
10423,46,1,Kolkata Knight Riders,Rajasthan Royals,13,2,DB Das,SC Ganguly,SK Warne,0,0,0,0,0,0,6,0,6,NaN,NaN,NaN
110786,466,2,Sunrisers Hyderabad,Royal Challengers Bangalore,14,3,KL Rahul,S Dhawan,SA Abbott,0,1,0,0,0,0,0,1,1,NaN,NaN,NaN
18287,79,2,Kolkata Knight Riders,Mumbai Indians,8,1,BJ Hodge,MN van Wyk,AM Nayar,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
16252,71,1,Kings XI Punjab,Rajasthan Royals,14,1,KC Sangakkara,IK Pathan,AD Mascarenhas,0,0,0,0,0,0,4,0,4,NaN,NaN,NaN
69742,295,2,Kings XI Punjab,Rajasthan Royals,2,5,N Saini,SE Marsh,SR Watson,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN


In [5]:
total_runs_df = deliveries.groupby(['match_id','inning'])['total_runs'].sum().reset_index()
total_runs_df = total_runs_df[total_runs_df['inning']==1]
total_runs_df.sample(10)


,match_id,inning,total_runs
1038,515,1,180
909,451,1,154
54,28,1,187
14,8,1,208
364,181,1,163
288,143,1,161
629,314,1,126
444,221,1,205
1030,511,1,199
1098,545,1,154


In [6]:
merged_df = total_runs_df.merge(matches,left_on='match_id',right_on='id').drop(columns=['player_of_match','umpire1','umpire2','umpire3','toss_winner','toss_decision','date','season','venue','id'])
merged_df = merged_df[merged_df['dl_applied']==0]
merged_df = merged_df[merged_df['result']=='normal']
merged_df.drop(columns=['result','dl_applied','inning'],inplace=True)
print(merged_df.shape)
merged_df.sample(5)

(553, 8)


,match_id,total_runs,city,team1,team2,winner,win_by_runs,win_by_wickets
439,440,138,Ranchi,Chennai Super Kings,Royal Challengers Bangalore,Royal Challengers Bangalore,0,5
473,474,156,Ahmedabad,Chennai Super Kings,Rajasthan Royals,Rajasthan Royals,0,8
173,174,82,Mumbai,Deccan Chargers,Royal Challengers Bangalore,Royal Challengers Bangalore,0,9
553,554,177,Visakhapatnam,Sunrisers Hyderabad,Mumbai Indians,Sunrisers Hyderabad,85,0
119,120,190,Chennai,Deccan Chargers,Chennai Super Kings,Deccan Chargers,31,0


In [7]:
merged_df2 = merged_df.merge(deliveries,left_on='match_id',right_on='match_id')
merged_df2['Current_Runs'] = merged_df2.groupby(['match_id','inning'])['total_runs_y'].cumsum()
print(merged_df2.shape)
merged_df2.head(5)

(132188, 29)


,match_id,total_runs_x,city,team1,team2,winner,win_by_runs,win_by_wickets,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs_y,player_dismissed,dismissal_kind,fielder,Current_Runs
0,1,222,Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,140,0,1,Kolkata Knight Riders,Royal Challengers Bangalore,1,1,SC Ganguly,BB McCullum,P Kumar,0,0,0,1,0,0,0,1,1,NaN,NaN,NaN,1
1,1,222,Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,140,0,1,Kolkata Knight Riders,Royal Challengers Bangalore,1,2,BB McCullum,SC Ganguly,P Kumar,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,1
2,1,222,Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,140,0,1,Kolkata Knight Riders,Royal Challengers Bangalore,1,3,BB McCullum,SC Ganguly,P Kumar,0,1,0,0,0,0,0,1,1,NaN,NaN,NaN,2
3,1,222,Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,140,0,1,Kolkata Knight Riders,Royal Challengers Bangalore,1,4,BB McCullum,SC Ganguly,P Kumar,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,2
4,1,222,Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,140,0,1,Kolkata Knight Riders,Royal Challengers Bangalore,1,5,BB McCullum,SC Ganguly,P Kumar,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,2


In [8]:
import warnings
warnings.filterwarnings(message='A value is trying to be set on a copy of a slice from a DataFrame.',action='ignore')
pd.options.mode.chained_assignment = None 
merged_inning2 = merged_df2[merged_df2['inning']==2]
merged_inning2['Required_Runs'] = merged_inning2['total_runs_x'] - merged_inning2['Current_Runs']
merged_inning2['Required_Runs'] = merged_inning2['Required_Runs']+1
print(merged_inning2.shape)
merged_inning2.head(10)

(64070, 30)


,match_id,total_runs_x,city,team1,team2,winner,win_by_runs,win_by_wickets,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs_y,player_dismissed,dismissal_kind,fielder,Current_Runs,Required_Runs
124,1,222,Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,140,0,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,1,R Dravid,W Jaffer,AB Dinda,0,0,0,0,0,0,1,0,1,NaN,NaN,NaN,1,222
125,1,222,Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,140,0,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,2,W Jaffer,R Dravid,AB Dinda,0,1,0,0,0,0,0,1,1,NaN,NaN,NaN,2,221
126,1,222,Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,140,0,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,3,W Jaffer,R Dravid,AB Dinda,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,2,221
127,1,222,Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,140,0,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,4,W Jaffer,R Dravid,AB Dinda,0,0,0,0,0,0,1,0,1,NaN,NaN,NaN,3,220
128,1,222,Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,140,0,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,5,R Dravid,W Jaffer,AB Dinda,0,0,0,0,0,0,1,0,1,NaN,NaN,NaN,4,219
129,1,222,Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,140,0,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,6,W Jaffer,R Dravid,AB Dinda,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,4,219
130,1,222,Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,140,0,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,7,W Jaffer,R Dravid,AB Dinda,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,4,219
131,1,222,Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,140,0,2,Royal Challengers Bangalore,Kolkata Knight Riders,2,1,R Dravid,W Jaffer,I Sharma,0,0,0,0,0,0,0,0,0,R Dravid,bowled,NaN,4,219
132,1,222,Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,140,0,2,Royal Challengers Bangalore,Kolkata Knight Riders,2,2,V Kohli,W Jaffer,I Sharma,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,4,219
133,1,222,Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,140,0,2,Royal Challengers Bangalore,Kolkata Knight Riders,2,3,V Kohli,W Jaffer,I Sharma,0,0,0,4,0,0,0,4,4,NaN,NaN,NaN,8,215


In [9]:

pd.options.mode.chained_assignment = None 
merged_inning2['player_dismissed']=merged_inning2['player_dismissed'].fillna(0)
merged_inning2['player_dismissed']=merged_inning2['player_dismissed'].apply(lambda x:0 if x==0 else 1)
merged_inning2.head(5)

,match_id,total_runs_x,city,team1,team2,winner,win_by_runs,win_by_wickets,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs_y,player_dismissed,dismissal_kind,fielder,Current_Runs,Required_Runs
124,1,222,Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,140,0,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,1,R Dravid,W Jaffer,AB Dinda,0,0,0,0,0,0,1,0,1,0,NaN,NaN,1,222
125,1,222,Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,140,0,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,2,W Jaffer,R Dravid,AB Dinda,0,1,0,0,0,0,0,1,1,0,NaN,NaN,2,221
126,1,222,Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,140,0,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,3,W Jaffer,R Dravid,AB Dinda,0,0,0,0,0,0,0,0,0,0,NaN,NaN,2,221
127,1,222,Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,140,0,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,4,W Jaffer,R Dravid,AB Dinda,0,0,0,0,0,0,1,0,1,0,NaN,NaN,3,220
128,1,222,Bangalore,Kolkata Knight Riders,Royal Challengers Bangalore,Kolkata Knight Riders,140,0,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,5,R Dravid,W Jaffer,AB Dinda,0,0,0,0,0,0,1,0,1,0,NaN,NaN,4,219


In [10]:
pd.options.mode.chained_assignment = None 
merged_inning2['current_balls_bowled'] = ((merged_inning2['over']-1)*6)+merged_inning2['ball']
merged_inning2['balls_left'] = 120 - merged_inning2['current_balls_bowled']
merged_inning2['current_run_rate'] = merged_inning2['Current_Runs']/((merged_inning2['current_balls_bowled']/6))
merged_inning2['required_run_rate'] = merged_inning2['Required_Runs']/((merged_inning2['balls_left']/6))
merged_inning2.drop(columns=['dismissal_kind','fielder','win_by_runs',	'win_by_wickets',	'team1','team2','batsman','non_striker','bowler','is_super_over','wide_runs','bye_runs','legbye_runs','noball_runs','penalty_runs','batsman_runs','extra_runs'],axis=1,inplace=True)
merged_inning2.head(10)

,match_id,total_runs_x,city,winner,inning,batting_team,bowling_team,over,ball,total_runs_y,player_dismissed,Current_Runs,Required_Runs,current_balls_bowled,balls_left,current_run_rate,required_run_rate
124,1,222,Bangalore,Kolkata Knight Riders,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,1,1,0,1,222,1,119,6.000000,11.193277
125,1,222,Bangalore,Kolkata Knight Riders,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,2,1,0,2,221,2,118,6.000000,11.237288
126,1,222,Bangalore,Kolkata Knight Riders,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,3,0,0,2,221,3,117,4.000000,11.333333
127,1,222,Bangalore,Kolkata Knight Riders,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,4,1,0,3,220,4,116,4.500000,11.379310
128,1,222,Bangalore,Kolkata Knight Riders,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,5,1,0,4,219,5,115,4.800000,11.426087
129,1,222,Bangalore,Kolkata Knight Riders,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,6,0,0,4,219,6,114,4.000000,11.526316
130,1,222,Bangalore,Kolkata Knight Riders,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,7,0,0,4,219,7,113,3.428571,11.628319
131,1,222,Bangalore,Kolkata Knight Riders,2,Royal Challengers Bangalore,Kolkata Knight Riders,2,1,0,1,4,219,7,113,3.428571,11.628319
132,1,222,Bangalore,Kolkata Knight Riders,2,Royal Challengers Bangalore,Kolkata Knight Riders,2,2,0,0,4,219,8,112,3.000000,11.732143
133,1,222,Bangalore,Kolkata Knight Riders,2,Royal Challengers Bangalore,Kolkata Knight Riders,2,3,4,0,8,215,9,111,5.333333,11.621622


In [11]:
merged_inning2['win'] = merged_inning2.apply(
    lambda row: 1 if row['winner'] == row['batting_team'] else 0, axis=1
)
print(merged_inning2.shape)
merged_inning2.sample(5)

(64070, 18)


,match_id,total_runs_x,city,winner,inning,batting_team,bowling_team,over,ball,total_runs_y,player_dismissed,Current_Runs,Required_Runs,current_balls_bowled,balls_left,current_run_rate,required_run_rate,win
46103,201,172,Hyderabad,Mumbai Indians,2,Deccan Chargers,Mumbai Indians,1,3,4,0,5,168,3,117,10.000000,8.615385,0
109159,471,183,Mumbai,Chennai Super Kings,2,Chennai Super Kings,Mumbai Indians,17,2,4,0,178,6,98,22,10.897959,1.636364,1
35638,156,154,Chandigarh,Kings XI Punjab,2,Kings XI Punjab,Mumbai Indians,4,5,1,0,40,115,23,97,10.434783,7.113402,1
97198,421,152,Delhi,Rajasthan Royals,2,Rajasthan Royals,Delhi Daredevils,7,2,1,0,41,112,38,82,6.473684,8.195122,1
28403,125,92,Bangalore,Royal Challengers Bangalore,2,Royal Challengers Bangalore,Rajasthan Royals,10,5,1,0,89,4,59,61,9.050847,0.393443,1


In [12]:
pd.options.mode.chained_assignment = None 
temp = merged_inning2.groupby('match_id')['player_dismissed'].cumsum().reset_index()

In [13]:
temp.head(20)

,index,player_dismissed
0,124,0
1,125,0
2,126,0
3,127,0
4,128,0
5,129,0
6,130,0
7,131,1
8,132,1
9,133,1


In [14]:
merged_inning2['index'] = [x for x in range(0,merged_inning2.shape[0])]
merged_inning2.head(20)

,match_id,total_runs_x,city,winner,inning,batting_team,bowling_team,over,ball,total_runs_y,player_dismissed,Current_Runs,Required_Runs,current_balls_bowled,balls_left,current_run_rate,required_run_rate,win,index
124,1,222,Bangalore,Kolkata Knight Riders,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,1,1,0,1,222,1,119,6.000000,11.193277,0,0
125,1,222,Bangalore,Kolkata Knight Riders,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,2,1,0,2,221,2,118,6.000000,11.237288,0,1
126,1,222,Bangalore,Kolkata Knight Riders,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,3,0,0,2,221,3,117,4.000000,11.333333,0,2
127,1,222,Bangalore,Kolkata Knight Riders,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,4,1,0,3,220,4,116,4.500000,11.379310,0,3
128,1,222,Bangalore,Kolkata Knight Riders,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,5,1,0,4,219,5,115,4.800000,11.426087,0,4
129,1,222,Bangalore,Kolkata Knight Riders,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,6,0,0,4,219,6,114,4.000000,11.526316,0,5
130,1,222,Bangalore,Kolkata Knight Riders,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,7,0,0,4,219,7,113,3.428571,11.628319,0,6
131,1,222,Bangalore,Kolkata Knight Riders,2,Royal Challengers Bangalore,Kolkata Knight Riders,2,1,0,1,4,219,7,113,3.428571,11.628319,0,7
132,1,222,Bangalore,Kolkata Knight Riders,2,Royal Challengers Bangalore,Kolkata Knight Riders,2,2,0,0,4,219,8,112,3.000000,11.732143,0,8
133,1,222,Bangalore,Kolkata Knight Riders,2,Royal Challengers Bangalore,Kolkata Knight Riders,2,3,4,0,8,215,9,111,5.333333,11.621622,0,9


In [15]:
merged_inning2.set_index('index',inplace=True)
merged_inning2.drop(columns=['player_dismissed'],axis=1,inplace=True)
merged_inning2.head(20)

,match_id,total_runs_x,city,winner,inning,batting_team,bowling_team,over,ball,total_runs_y,Current_Runs,Required_Runs,current_balls_bowled,balls_left,current_run_rate,required_run_rate,win
index,,,,,,,,,,,,,,,,,
0,1,222,Bangalore,Kolkata Knight Riders,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,1,1,1,222,1,119,6.000000,11.193277,0
1,1,222,Bangalore,Kolkata Knight Riders,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,2,1,2,221,2,118,6.000000,11.237288,0
2,1,222,Bangalore,Kolkata Knight Riders,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,3,0,2,221,3,117,4.000000,11.333333,0
3,1,222,Bangalore,Kolkata Knight Riders,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,4,1,3,220,4,116,4.500000,11.379310,0
4,1,222,Bangalore,Kolkata Knight Riders,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,5,1,4,219,5,115,4.800000,11.426087,0
5,1,222,Bangalore,Kolkata Knight Riders,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,6,0,4,219,6,114,4.000000,11.526316,0
6,1,222,Bangalore,Kolkata Knight Riders,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,7,0,4,219,7,113,3.428571,11.628319,0
7,1,222,Bangalore,Kolkata Knight Riders,2,Royal Challengers Bangalore,Kolkata Knight Riders,2,1,0,4,219,7,113,3.428571,11.628319,0
8,1,222,Bangalore,Kolkata Knight Riders,2,Royal Challengers Bangalore,Kolkata Knight Riders,2,2,0,4,219,8,112,3.000000,11.732143,0


In [16]:
data = pd.concat([temp,merged_inning2],axis=1)
final_df = data
final_df.rename(columns={'player_dismissed':'wickets_down','total_runs_x':'target_score'},inplace=True)
columns = final_df.columns
print(columns)

Index(['index', 'wickets_down', 'match_id', 'target_score', 'city', 'winner',
       'inning', 'batting_team', 'bowling_team', 'over', 'ball',
       'total_runs_y', 'Current_Runs', 'Required_Runs', 'current_balls_bowled',
       'balls_left', 'current_run_rate', 'required_run_rate', 'win'],
      dtype='object')


In [17]:
final_df.head(20)

,index,wickets_down,match_id,target_score,city,winner,inning,batting_team,bowling_team,over,ball,total_runs_y,Current_Runs,Required_Runs,current_balls_bowled,balls_left,current_run_rate,required_run_rate,win
0,124,0,1,222,Bangalore,Kolkata Knight Riders,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,1,1,1,222,1,119,6.000000,11.193277,0
1,125,0,1,222,Bangalore,Kolkata Knight Riders,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,2,1,2,221,2,118,6.000000,11.237288,0
2,126,0,1,222,Bangalore,Kolkata Knight Riders,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,3,0,2,221,3,117,4.000000,11.333333,0
3,127,0,1,222,Bangalore,Kolkata Knight Riders,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,4,1,3,220,4,116,4.500000,11.379310,0
4,128,0,1,222,Bangalore,Kolkata Knight Riders,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,5,1,4,219,5,115,4.800000,11.426087,0
5,129,0,1,222,Bangalore,Kolkata Knight Riders,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,6,0,4,219,6,114,4.000000,11.526316,0
6,130,0,1,222,Bangalore,Kolkata Knight Riders,2,Royal Challengers Bangalore,Kolkata Knight Riders,1,7,0,4,219,7,113,3.428571,11.628319,0
7,131,1,1,222,Bangalore,Kolkata Knight Riders,2,Royal Challengers Bangalore,Kolkata Knight Riders,2,1,0,4,219,7,113,3.428571,11.628319,0
8,132,1,1,222,Bangalore,Kolkata Knight Riders,2,Royal Challengers Bangalore,Kolkata Knight Riders,2,2,0,4,219,8,112,3.000000,11.732143,0
9,133,1,1,222,Bangalore,Kolkata Knight Riders,2,Royal Challengers Bangalore,Kolkata Knight Riders,2,3,4,8,215,9,111,5.333333,11.621622,0


In [18]:
final_df.columns

Index(['index', 'wickets_down', 'match_id', 'target_score', 'city', 'winner',
       'inning', 'batting_team', 'bowling_team', 'over', 'ball',
       'total_runs_y', 'Current_Runs', 'Required_Runs', 'current_balls_bowled',
       'balls_left', 'current_run_rate', 'required_run_rate', 'win'],
      dtype='object')

In [19]:
final_df.drop(columns=['index','match_id','winner','inning','total_runs_y'],axis=1,inplace=True)
final_df.head(10)

,wickets_down,target_score,city,batting_team,bowling_team,over,ball,Current_Runs,Required_Runs,current_balls_bowled,balls_left,current_run_rate,required_run_rate,win
0,0,222,Bangalore,Royal Challengers Bangalore,Kolkata Knight Riders,1,1,1,222,1,119,6.000000,11.193277,0
1,0,222,Bangalore,Royal Challengers Bangalore,Kolkata Knight Riders,1,2,2,221,2,118,6.000000,11.237288,0
2,0,222,Bangalore,Royal Challengers Bangalore,Kolkata Knight Riders,1,3,2,221,3,117,4.000000,11.333333,0
3,0,222,Bangalore,Royal Challengers Bangalore,Kolkata Knight Riders,1,4,3,220,4,116,4.500000,11.379310,0
4,0,222,Bangalore,Royal Challengers Bangalore,Kolkata Knight Riders,1,5,4,219,5,115,4.800000,11.426087,0
5,0,222,Bangalore,Royal Challengers Bangalore,Kolkata Knight Riders,1,6,4,219,6,114,4.000000,11.526316,0
6,0,222,Bangalore,Royal Challengers Bangalore,Kolkata Knight Riders,1,7,4,219,7,113,3.428571,11.628319,0
7,1,222,Bangalore,Royal Challengers Bangalore,Kolkata Knight Riders,2,1,4,219,7,113,3.428571,11.628319,0
8,1,222,Bangalore,Royal Challengers Bangalore,Kolkata Knight Riders,2,2,4,219,8,112,3.000000,11.732143,0
9,1,222,Bangalore,Royal Challengers Bangalore,Kolkata Knight Riders,2,3,8,215,9,111,5.333333,11.621622,0


In [20]:
final_df.shape

(64070, 14)

Now before moving any further, let's check if our data has some missing values or some infinite values, it is necessary to remove them as it will cause difficulties in future model training

In [21]:
final_df.isnull().sum()

wickets_down              0
target_score              0
city                    832
batting_team              0
bowling_team              0
over                      0
ball                      0
Current_Runs              0
Required_Runs             0
current_balls_bowled      0
balls_left                0
current_run_rate          0
required_run_rate         5
win                       0
dtype: int64

In [22]:
final_df.dropna(inplace=True)
final_df.describe()

C:\Users\moham\AppData\Roaming\Python\Python312\site-packages\numpy\core\_methods.py:49: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


,wickets_down,target_score,over,ball,Current_Runs,Required_Runs,current_balls_bowled,balls_left,current_run_rate,required_run_rate,win
count,63233.000000,63233.000000,63233.000000,63233.000000,63233.000000,63233.000000,63233.000000,63233.000000,63233.000000,6.323300e+04,63233.000000
mean,2.459886,161.579460,9.916768,3.610251,70.884285,91.695175,57.110860,62.889140,7.207274,NaN,0.524853
std,2.152531,28.593372,5.552802,1.808843,45.432587,48.996906,33.328396,33.328396,2.197960,NaN,0.499386
min,0.000000,67.000000,1.000000,1.000000,0.000000,-5.000000,1.000000,-2.000000,0.000000,-inf,0.000000
25%,1.000000,142.000000,5.000000,2.000000,33.000000,53.000000,28.000000,35.000000,6.052174,7.090909e+00,0.000000
50%,2.000000,163.000000,10.000000,4.000000,68.000000,91.000000,56.000000,64.000000,7.250000,8.812500e+00,1.000000
75%,4.000000,181.000000,15.000000,5.000000,105.000000,128.000000,85.000000,92.000000,8.388889,1.084337e+01,1.000000
max,10.000000,263.000000,20.000000,9.000000,223.000000,264.000000,122.000000,119.000000,36.000000,inf,1.000000


In [23]:
final_df= final_df[final_df['balls_left']!=0]
final_df.describe()

,wickets_down,target_score,over,ball,Current_Runs,Required_Runs,current_balls_bowled,balls_left,current_run_rate,required_run_rate,win
count,63026.000000,63026.000000,63026.000000,63026.000000,63026.000000,63026.000000,63026.000000,63026.000000,63026.000000,63026.000000,63026.000000
mean,2.444055,161.538492,9.883651,3.602402,70.615730,91.922762,56.904309,63.095691,7.205877,10.451581,0.526307
std,2.136386,28.596313,5.531714,1.806610,45.245475,48.903504,33.187310,33.187310,2.200457,14.460844,0.499311
min,0.000000,67.000000,1.000000,1.000000,0.000000,-5.000000,1.000000,-2.000000,0.000000,-516.000000,0.000000
25%,1.000000,142.000000,5.000000,2.000000,33.000000,53.000000,28.000000,35.000000,6.000000,7.090909,0.000000
50%,2.000000,163.000000,10.000000,4.000000,67.000000,91.000000,56.000000,64.000000,7.246753,8.804348,1.000000
75%,4.000000,181.000000,15.000000,5.000000,104.000000,128.000000,85.000000,92.000000,8.389381,10.800000,1.000000
max,10.000000,263.000000,20.000000,9.000000,222.000000,264.000000,122.000000,119.000000,36.000000,792.000000,1.000000


In [24]:
print(final_df.shape)
final_df.isnull().sum()

(63026, 14)


wickets_down            0
target_score            0
city                    0
batting_team            0
bowling_team            0
over                    0
ball                    0
Current_Runs            0
Required_Runs           0
current_balls_bowled    0
balls_left              0
current_run_rate        0
required_run_rate       0
win                     0
dtype: int64

In [25]:
final_df.sample()

,wickets_down,target_score,city,batting_team,bowling_team,over,ball,Current_Runs,Required_Runs,current_balls_bowled,balls_left,current_run_rate,required_run_rate,win
44573,8,148,Kolkata,Chennai Super Kings,Mumbai Indians,15,4,82,67,88,32,5.590909,12.5625,0


Now preprocessing is completely done, and we are good to go. Data has been completely reduced to the form it should be in. Now data has all required columns which are necessary for the model training. Now we'll apply ml processings and functions on it to have a good model

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [27]:
X_train,X_test,y_train,y_test = train_test_split(final_df.drop(columns=['win'],axis=1),final_df['win'],random_state=0,test_size=0.2)
print(f"X_train shape : {X_train.shape}\nX_test shape : {X_test.shape}\ny_train shape : {y_train.shape}\ny_test shape : {y_test.shape}")

X_train shape : (50420, 13)
X_test shape : (12606, 13)
y_train shape : (50420,)
y_test shape : (12606,)


In [28]:
trf = ColumnTransformer([('OneHotEncoding',OneHotEncoder(drop='first',sparse_output=False),['city','batting_team','bowling_team'])],remainder='passthrough')

pipe = Pipeline(steps=[('encoding',trf),('model_training',LogisticRegression(solver='liblinear'))])

In [29]:
pipe.fit(X_train,y_train)


Pipeline(steps=[('encoding',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('OneHotEncoding',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['city', 'batting_team',
                                                   'bowling_team'])])),
                ('model_training', LogisticRegression(solver='liblinear'))])

In [30]:
ypred = pipe.predict(X_test)
print('accuracy score is : ',int(accuracy_score(ypred,y_test)*100),'%')

accuracy score is :  82 %


Now all we needed to do with the data has been done. We have cleaned the data and also trained an ml model, it's time to convert it into a web app. For that I'll use pickle to dump my ml model object in a new file and we'll work there. ^_^

In [31]:
import pickle
pickle.dump(pipe,open('ipl.pkl','wb'))

In [32]:
final_df['bowling_team'].unique()

array(['Kolkata Knight Riders', 'Chennai Super Kings', 'Rajasthan Royals',
       'Mumbai Indians', 'Deccan Chargers', 'Kings XI Punjab',
       'Royal Challengers Bangalore', 'Delhi Daredevils',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Sunrisers Hyderabad',
       'Rising Pune Supergiants', 'Gujarat Lions'], dtype=object)

In [33]:
final_df['batting_team'].unique()

array(['Royal Challengers Bangalore', 'Kings XI Punjab',
       'Delhi Daredevils', 'Kolkata Knight Riders', 'Rajasthan Royals',
       'Mumbai Indians', 'Chennai Super Kings', 'Deccan Chargers',
       'Pune Warriors', 'Kochi Tuskers Kerala', 'Sunrisers Hyderabad',
       'Rising Pune Supergiants', 'Gujarat Lions'], dtype=object)

In [34]:
final_df['city'].unique()

array(['Bangalore', 'Chandigarh', 'Delhi', 'Mumbai', 'Kolkata', 'Jaipur',
       'Hyderabad', 'Chennai', 'Cape Town', 'Port Elizabeth', 'Durban',
       'Centurion', 'East London', 'Johannesburg', 'Kimberley',
       'Bloemfontein', 'Ahmedabad', 'Cuttack', 'Nagpur', 'Dharamsala',
       'Kochi', 'Indore', 'Visakhapatnam', 'Pune', 'Raipur', 'Ranchi',
       'Abu Dhabi', 'Sharjah', 'Rajkot', 'Kanpur'], dtype=object)